In [ ]:
#Import libraries

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from flask import Flask, request, jsonify
from keras.preprocessing import image
from keras.models import load_model
from keras.models import save_model
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
def load_data(data_dir):
    images = []
    labels = []

    for filename in os.listdir(os.path.join(data_dir, "images")):
        if filename.endswith('.jpg'):
            img_path = os.path.join(data_dir, "images", filename)
            label_path = os.path.join(data_dir, "labels", filename.replace('.jpg', '.txt'))

            if os.path.isfile(label_path):
                with open(label_path, 'r') as label_file:
                    line = label_file.readline().strip()

                    if not line:
                        print(f"Empty line in file: {label_path}")
                        continue
                        
                    try:
                        label = int(line.split()[0])
                        labels.append(label)
                    except (ValueError, IndexError) as e:
                        print(f"Error processing line in file {label_path}: {line}")
                        print(f"Error details: {e}")

                # Resize the image to the expected input shape (224, 224)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (224, 224))
                images.append(img)

    return np.array(images), np.array(labels)

# Specify your dataset directory
dataset_dir = '/Users/gurl/Desktop/BrainTumorImages/TumorDetectionYolov8/OD8/Brain Tumor Detection/train'

# Load the data
images, labels = load_data(dataset_dir)

In [ ]:
# Split dataset into training and testing sets

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Ensure labels start from 0
min_label = np.min(y_train)
y_train = y_train - min_label
y_test = y_test - min_label

# Count the number of unique labels
num_classes = len(np.unique(y_train))

# Convert labels to categorical format
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

In [ ]:
# Load the pre-trained ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train_categorical, epochs=5, batch_size=30, validation_data=(x_test, y_test_categorical))

# Evaluate the model on the test set
accuracy = model.evaluate(x_test, y_test_categorical)[1]
print(f'Test Accuracy: {accuracy}')